In [8]:
from __future__ import division
import csv,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time, locale
from IPython import display as d
import nvd3

reload(sys)  # Reload does the trick!
sys.setdefaultencoding('UTF8')

nvd3.ipynb.initialize_javascript(use_remote=True)

locale.setlocale(locale.LC_ALL, 'es_ES')
locale.setlocale(locale.LC_ALL, 'C')
locale.format("%10.2f", 1234.567, grouping=True) 
pd.options.display.float_format = '{:,.2f}'.format

# Static files
FILE_SUBVENCIONES = 'data/datos_subvenciones.csv'
FILE_AYUDAS = 'data/ayudas.json'
FILE_ELECCIONES = 'data/04_201105_1.csv'
FILE_MUNICIPIOS = 'data/smex99.csv'

def totimestamp(dt, epoch=datetime(1970,1,1)):
    td = dt - epoch
    ttime = (td.microseconds + (td.seconds + td.days * 86400) * 10**6) / 10**6 
    return ttime * 1000

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
#Read files
subvenciones = pd.read_csv(FILE_SUBVENCIONES, decimal=',')
ayudas = pd.read_json(FILE_AYUDAS)
municipios = pd.read_csv(FILE_MUNICIPIOS, quotechar='"', sep=';', decimal=',', thousands='.')
elecciones = pd.read_csv(FILE_ELECCIONES, quotechar='"', sep=';', decimal=',', thousands='.')

El importe total de las subvenciones includas en los datos abiertos es:

In [10]:
subvenciones['importe'].sum()

1028817129.8299986

In [11]:
subvenciones['mes_concesion'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y%m'))
subvenciones['fecha_concesion_tmp'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
subvenciones['mes_concesion_timestamp'] = subvenciones['fecha_concesion'].apply(lambda x: totimestamp(datetime(datetime.fromtimestamp(x).year, datetime.fromtimestamp(x).month,1,0,0)))

In [12]:
subvenciones_por_fecha = subvenciones.groupby('fecha_concesion').sum()
subvenciones_por_mes = subvenciones.groupby('mes_concesion_timestamp').sum()
subvenciones_por_mes_finalidad = subvenciones.groupby(['mes_concesion_timestamp','finalidad']).sum()

In [13]:
type = 'stackedAreaChart'
chart2 = nvd3.stackedAreaChart(name=type,height=450,width=850,use_interactive_guideline=True, x_is_date=True)
#nb_element = 50
#xdata = range(len(list(subvenciones_por_mes.index)))
xdata = list(subvenciones_por_mes.index)
ydata = list(subvenciones_por_mes['importe'])
chart2.add_serie(name="Ejecución Subvenciones", y=ydata, x=xdata)
chart2.margin_left = 100
chart2

In [14]:
type = 'stackedAreaChart'
chart3 = nvd3.stackedAreaChart(name=type,height=450,width=850,use_interactive_guideline=True, x_is_date=True)
xdata = list(subvenciones_por_mes_finalidad.index)
ydata = list(subvenciones_por_mes_finalidad['importe'])
chart3.add_serie(name="Ejecución Subvenciones", y=ydata, x=xdata)
chart3.margin_left = 100
chart3

In [15]:
elecciones['Nombre de Comunidad'] = elecciones['Nombre de Comunidad'].apply(lambda x: x.rstrip())
filas_andalucia = elecciones['Nombre de Comunidad'] == 'Andaluc\xc3\xada' #Andalucía
#cols_elecciones_significativas = (elecciones[rows_andalucia].sum() > 1000)
#resumen_elecciones_andalucia = elecciones[filas_andalucia][elecciones.columns[cols_elecciones_significativas]]

In [16]:
subvenciones_por_finalidad = subvenciones.groupby('finalidad').sum().sort('importe',ascending=0).reset_index()

In [17]:
subvenciones_por_finalidad[subvenciones_por_finalidad.importe > 1000000L]

,finalidad,id,importe,fecha_concesion,mes_concesion_timestamp
0,FOMENTO DEL EMPLEO,166470517,"326,664,161.05",5031910407600,"5,026,605,724,800,000.00"
1,IMPULSO CONSTRUCCION SOSTENIBL,131279303,"47,413,570.45",5677701422400,"5,672,810,160,000,000.00"
2,PTMOS OBRAS Y SERVICIOS CC.LL.,130163,"44,313,884.42",11232374400,"11,212,646,400,000.00"
3,CREACION Y CONSOL. RED AGENCIA,65180,"41,465,080.00",2812050000,"2,808,432,000,000.00"
4,OBRAS DE REGADIOS,2925677,"38,821,799.19",83320110000,"83,191,276,800,000.00"
5,INVESTIGACION CIENT�FICA,4812112,"37,125,304.04",250099102800,"249,900,508,800,000.00"
6,COOPERACI�N PARA EL DESARROLLO,12314050,"35,348,320.64",247998078000,"247,898,534,400,000.00"
7,MODERNIZACION EXPL. AGRARIAS,68068992,"29,605,950.68",1604478060000,"1,603,936,944,000,000.00"
8,GASTOS DE FUNCIONAMIENTO,2955704,"29,168,009.56",95874937200,"95,731,718,400,000.00"
9,INV.INSTAL.APROVECH.ENERG.RENO,250194549,"23,680,237.58",17622890344800,"17,608,520,534,400,000.00"
